In [1]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3

C:\Users\diego\AppData\Local\Temp\ipykernel_28152\1521701836.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
DB = {'servername': 'LAPTOP-3S422SS6\\SQLEXPRESS',
'database': 'DAD'}

export_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB['servername'] +
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

In [4]:
Crm_connection= sqlite3.connect("go_crm.sqlite")

In [13]:
newQuery = "SELECT * FROM sales_demographic"
salesdemographic = pd.read_sql(newQuery,Crm_connection)
newQuery = "SELECT * FROM age_group"
agegroup = pd.read_sql(newQuery,Crm_connection)
newQuery = "SELECT * FROM retailer"
retailer = pd.read_sql(newQuery,Crm_connection)

In [16]:
agegroup['age_group'] = agegroup['LOWER_AGE'] + '-'+ agegroup['UPPER_AGE']
salesdemographic

,DEMOGRAPHIC_CODE,RETAILER_CODEMR,AGE_GROUP_CODE,SALES_PERCENT,TRIAL222
0,1,1,1,52,T
1,10,2,4,16,T
2,100,17,4,11,T
3,1000,164,3,6,T
4,1001,164,4,9,T
...,...,...,...,...,...
2479,995,143,4,4,T
2480,996,143,5,5,T
2481,997,143,6,3,T
2482,998,164,1,32,T


In [19]:
merge = pd.merge(salesdemographic, agegroup, left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE', how='left')
merge = pd.merge(merge, retailer, left_on='RETAILER_CODEMR', right_on='RETAILER_CODEMR', how='left')
merge = merge[['DEMOGRAPHIC_CODE', 'SALES_PERCENT', 'age_group', 'COMPANY_NAME',]]

In [21]:
merge_export_cursor = export_conn.cursor()

for index, row in merge.iterrows():
    if pd.notna(row['COMPANY_NAME']):  # Check if COMPANY_NAME is not NaN
        try:
            # Escape single quotes by replacing each single quote with two single quotes
            age_group = row['age_group'].replace("'", "''")
            company_name = row['COMPANY_NAME'].replace("'", "''")

            # Construct the INSERT query with escaped strings
            query = f"""
            INSERT INTO SALES_DEMOGRAPHIC
            (DEMOGRAPHIC_CODE, SALES_PERCENT, AGE_GROUP, COMPANY_NAME) 
            VALUES
            ({row['DEMOGRAPHIC_CODE']}, {row['SALES_PERCENT']}, '{age_group}', '{company_name}');
            """
            merge_export_cursor.execute(query)
        except pyodbc.Error as e:
            print("An error occurred:", e)
            print(query)  # This will print the failing SQL query
    else:
        # Optionally, print a message or handle rows with NaN COMPANY_NAME differently
        print(f"Skipping row {index} due to NaN COMPANY_NAME")

export_conn.commit()  # Commit the transactions
merge_export_cursor.close()  # Close the cursor

Skipping row 2 due to NaN COMPANY_NAME
Skipping row 3 due to NaN COMPANY_NAME
Skipping row 4 due to NaN COMPANY_NAME
Skipping row 5 due to NaN COMPANY_NAME
Skipping row 6 due to NaN COMPANY_NAME
Skipping row 7 due to NaN COMPANY_NAME
Skipping row 8 due to NaN COMPANY_NAME
Skipping row 9 due to NaN COMPANY_NAME
Skipping row 10 due to NaN COMPANY_NAME
Skipping row 11 due to NaN COMPANY_NAME
Skipping row 12 due to NaN COMPANY_NAME
Skipping row 13 due to NaN COMPANY_NAME
Skipping row 14 due to NaN COMPANY_NAME
Skipping row 15 due to NaN COMPANY_NAME
Skipping row 16 due to NaN COMPANY_NAME
Skipping row 17 due to NaN COMPANY_NAME
Skipping row 18 due to NaN COMPANY_NAME
Skipping row 19 due to NaN COMPANY_NAME
Skipping row 24 due to NaN COMPANY_NAME
Skipping row 33 due to NaN COMPANY_NAME
Skipping row 34 due to NaN COMPANY_NAME
Skipping row 35 due to NaN COMPANY_NAME
Skipping row 36 due to NaN COMPANY_NAME
Skipping row 37 due to NaN COMPANY_NAME
Skipping row 38 due to NaN COMPANY_NAME
Skipping